In [11]:
import requests
from datetime import datetime
import pandas as pd
import json
import tweepy
import os
from pathlib import Path

## On récupère la date du jour

In [12]:
today = datetime.now()
yesterday = today - pd.Timedelta(days=1)

## Quels sont les fichiers publiés en open data aujourd'hui ?

In [13]:
def retrieve_files_date(date):
    formated_date = date.strftime('%Y-%m-%d')
    try:
        return(pd
            .read_csv(
                f'https://www.assemblee-nationale.fr/dyn/opendata/list-publication/publication_{formated_date}.csv', 
                sep=';', 
                names=['date', 'url']
            )
        )
    except:
        return(pd.DataFrame(columns=['date', 'url']))

files = pd.concat([retrieve_files_date(date) for date in [yesterday, today]], ignore_index=True)

## On isole tous les rapports d'information

- L'id du rapport permet d'accéder au fichier de description json 
- Si l'id est différent du full_id, cela veut dire que c'est un rapport en plusieurs parties. Il faudra trouver la correspondance dans le fichier de description json

In [4]:
ris = (files
    # les pdf d'un rapport d'information
    .loc[files['url'].str.contains(r'RIN.+\.pdf$')]    
    .assign(
        full_id = lambda x: x['url'].str.extract(pat = '(RIN.+)\.pdf')        
    )
)

# https://www.codegrepper.com/code-examples/javascript/regex+match+anything+except+character
ris[['id', 'tome']] =  ris['full_id'].str.extract(pat = '(RIN[^-]+)-?(.*)', expand=True)

ris

,date,url,full_id,id,tome
336,2022-01-24 12:35:55,https://www.assemblee-nationale.fr/dyn/opendat...,RINFANR5L15B4790,RINFANR5L15B4790,
704,2022-01-24 17:14:51,https://www.assemblee-nationale.fr/dyn/opendat...,RINFANR5L15B4915,RINFANR5L15B4915,


In [16]:
raps = (files
    # les pdf d'un rapport
    .loc[files['url'].str.contains(r'RAPP.+\.pdf$')]    
    .assign(
        full_id = lambda x: x['url'].str.extract(pat = '(RAPP.+)\.pdf'),        
    )
)

# https://www.codegrepper.com/code-examples/javascript/regex+match+anything+except+character
raps[['id', 'tome']] =  raps['full_id'].str.extract(pat = '(RAPP[^-]+)-?(.*)', expand=True)

raps

,date,url,full_id,id,tome
295,2022-01-24 12:15:30,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4891,RAPPANR5L15B4891,
300,2022-01-24 12:16:08,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4891,RAPPANR5L15B4891,
429,2022-01-24 15:10:18,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4921,RAPPANR5L15B4921,


In [29]:
all_rapports = pd.concat([ris, raps], ignore_index=True).drop_duplicates(subset=['id', 'full_id'])

In [30]:
descriptions = {}
for id in all_rapports['id'].unique():
    url = f'https://www.assemblee-nationale.fr/dyn/opendata/{id}.json'
    response = requests.get(url)
    if response.status_code == 200:
        descriptions[id] = json.loads(response.text)
    else:
        print(f'{id} not found')

Pour le moment, on va juste mettre la description de base dans la df

In [31]:
def get_desc(id):
    return descriptions[id]['titres']['titrePrincipalCourt']

def get_type(id):
    if 'libelle' in descriptions[id]['classification']['sousType']:
        return(f"{descriptions[id]['classification']['type']['libelle']} {descriptions[id]['classification']['sousType']['libelle']}")
    else: 
        return descriptions[id]['classification']['type']['libelle']

all_rapports = all_rapports.assign(
    description = lambda x: x['id'].apply(get_desc),
    rapport_type = lambda x: x['id'].apply(get_type)
)

all_rapports

,date,url,full_id,id,tome,description,rapport_type
0,2022-01-24 12:15:30,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4891,RAPPANR5L15B4891,,Aménagement du Rhône,Rapport
2,2022-01-24 15:10:18,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4921,RAPPANR5L15B4921,,choix du nom issu de la filiation,Rapport


Quels sont les nouveaux RI ?   

In [32]:
try:
    old_rapports = pd.read_csv('rapports.csv', sep=';')
except:
    old_rapports = pd.DataFrame({'full_id': []})

new_rapports = all_rapports.loc[~all_rapports['full_id'].isin(old_rapports['full_id'])]    
new_rapports

,date,url,full_id,id,tome,description,rapport_type
0,2022-01-24 12:15:30,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4891,RAPPANR5L15B4891,,Aménagement du Rhône,Rapport
2,2022-01-24 15:10:18,https://www.assemblee-nationale.fr/dyn/opendat...,RAPPANR5L15B4921,RAPPANR5L15B4921,,choix du nom issu de la filiation,Rapport


## On sauvegarde les nouveaux rapports

In [34]:
pd.concat([old_rapports, new_rapports], ignore_index=True).to_csv('rapports.csv', sep=';', index=False)

## On télécharge les nouveaux fichiers

In [39]:
dirs_dict = {
    "Rapport d'information": "ris",
    "Rapport d'enquête": "res"
}
for index, row in new_rapports.iterrows():
    dir = dirs_dict[row['rapport_type'] ] if row['rapport_type'] in dirs_dict else 'raps'
    file = Path(f'../data/assnat/{dir}/{row["full_id"]}.pdf')
    if not file.exists():
        r = requests.get(row['url'])
        if r.status_code == 200:
            file.parent.mkdir(parents=True, exist_ok=True)    
            file.write_bytes(r.content)

## On tweete les nouveaux rapports

In [40]:
try:
    credentials = json.load(open('../twitter-credentials.json'))
except:
    # gh actions secrets
    credentials = {key: os.environ[key] for key in ["TWITTER_API_KEY", "TWITTER_API_SECRET", "TWITTER_ACCESS_KEY", "TWITTER_ACCESS_SECRET"]}

In [41]:
auth = tweepy.OAuthHandler(credentials['TWITTER_API_KEY'], credentials['TWITTER_API_SECRET'])
auth.set_access_token(credentials['TWITTER_ACCESS_KEY'], credentials['TWITTER_ACCESS_SECRET'])
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except Exception as e:
    print(e)
    print("Error during authentication")

Authentication OK


In [44]:
for index, row in new_rapports.iterrows():
    icons_dict = {
        "Rapport d'information": "ℹ️",
        "Rapport d'enquête": "🔍"
    }
    icon = icons_dict[row['rapport_type']] if row['rapport_type'] in icons_dict else '📖'
    tweet = f'{icon} • Nouveau {row["rapport_type"].lower()} {row["url"]} {row["description"]}'
    if len(tweet) > 280:
        tweet = tweet[0:277] + '...'    
    print(tweet)
    api.update_status(tweet)

📖 • Nouveau rapport https://www.assemblee-nationale.fr/dyn/opendata/RAPPANR5L15B4891.pdf Aménagement du Rhône
📖 • Nouveau rapport https://www.assemblee-nationale.fr/dyn/opendata/RAPPANR5L15B4921.pdf choix du nom issu de la filiation
